In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

URL = 'https://practicum-content.s3.us-west-1.amazonaws.com/data-analyst-eng/moved_chicago_weather_2017.html'
req = requests.get(URL)
soup = BeautifulSoup(req.text,'html.parser')

# print(soup)

table = soup.find("table", attrs={"id":"weather_records"})
# print(table)

heading_table = []
content = []

for row in table.find_all("th"):
    heading_table.append(row.text)
# print(heading_table)

for row in table.find_all("tr"):
    if not row.find_all("th"):
        content.append([element.text for element in row.find_all("td")])
# print(content)

weather_records = pd.DataFrame(content, columns=heading_table)
print(weather_records)

           Date and time Temperature       Description
0    2017-11-01 00:00:00     276.150     broken clouds
1    2017-11-01 01:00:00     275.700  scattered clouds
2    2017-11-01 02:00:00     275.610   overcast clouds
3    2017-11-01 03:00:00     275.350     broken clouds
4    2017-11-01 04:00:00     275.240     broken clouds
..                   ...         ...               ...
692  2017-11-29 20:00:00     281.340        few clouds
693  2017-11-29 21:00:00     281.690      sky is clear
694  2017-11-29 22:00:00     281.070        few clouds
695  2017-11-29 23:00:00     280.060      sky is clear
696  2017-11-30 00:00:00     278.460      sky is clear

[697 rows x 3 columns]


In [ ]:
-- 1.Imprima o campo company_name. 
-- Encontre o número de corridas de táxi para cada empresa de táxi de 15 a 16 de novembro de 2017, 
-- nomeie o campo resultante como trips_amount e imprima-o também.
--  Classifique os resultados pelo campo trips_amount em ordem decrescente.

SELECT
    cabs.company_name,
    COUNT(DISTINCT trips.trip_id) AS trips_amount
FROM
    cabs
INNER JOIN trips ON trips.cab_id = cabs.cab_id
WHERE
    CAST(trips.start_ts AS date) >= '2017-11-15'
    AND CAST(trips.start_ts AS date) <= '2017-11-16'
GROUP BY
    cabs.company_name
ORDER BY
    COUNT(DISTINCT trips.trip_id) DESC

In [ ]:
-- 2. Encontre o número de corridas para cada empresa de táxi cujo nome contém as palavras "Amarelo" ou "Azul" de 1 a 7 de novembro de 2017. 
-- Nomeie a variável resultante como trips_amount. Agrupe os resultados pelo campo company_name.

SELECT
    cabs.company_name as company_name,
    COUNT(DISTINCT trips.trip_id) as trips_amount
FROM
    cabs
INNER JOIN
    trips ON trips.cab_id = cabs.cab_id
WHERE
    CAST(trips.start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'
    AND cabs.company_name like '%%Yellow%%'
GROUP BY
    cabs.company_name
UNION ALL
SELECT
    cabs.company_name as company_name,
    COUNT(DISTINCT trips.trip_id) as trips_amount
FROM
    cabs
INNER JOIN
    trips ON trips.cab_id = cabs.cab_id
WHERE
    CAST(trips.start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'
    AND cabs.company_name like '%%Blue%%'
GROUP BY
    cabs.company_name

In [ ]:
-- 3. De 1 a 7 de novembro de 2017, as empresas de táxi mais populares foram Flash Cab e Taxi Affiliation Services. 
-- Encontre o número de corridas para essas duas empresas e nomeie a variável resultante como trips_amount. 
-- Junte as corridas de todas as outras empresas no grupo "Other". 
-- Agrupe os dados por nomes de empresas de táxi. 
-- Nomeie o campo com os nomes das empresas de táxi company. 
-- Ordene o resultado em ordem decrescente por trips_amount.

SELECT
    CASE
        WHEN cabs.company_name = 'Flash Cab' THEN 'Flash Cab'
        WHEN cabs.company_name = 'Taxi Affiliation Services' THEN 'Taxi Affiliation Services'
    ELSE
        'Other'
    END AS company,
    COUNT(trips.trip_id) as trips_amount
FROM
    cabs
INNER JOIN
    trips 
ON  
    trips.cab_id = cabs.cab_id
WHERE
    CAST(trips.start_ts AS date) BETWEEN '2017-11-01' AND '2017-11-07'
GROUP BY
    company
ORDER BY
    trips_amount DESC

In [ ]:
-- 4. Recupere os identificadores dos bairros O'Hare e Loop da tabela neighborhoods.

SELECT
    name,
    neighborhood_id
FROM neighborhoods
WHERE 
    neighborhoods.name like '%%Hare%%'
    OR neighborhoods.name = 'Loop'

In [ ]:
-- 5. Para cada hora, recupere os registros de condições climáticas da tabela weather_records. 
-- Usando o operador CASE, divida todas as horas em dois grupos: 
-- Bad se o campo descrição contiver as palavras rain ou storm e Good para outros. 
-- Nomeie o campo resultante como weather_conditions. 
-- A tabela final deve incluir dois campos: data e hora (ts) e weather_conditions.

SELECT
    weather_records.ts as ts,
    CASE
        WHEN weather_records.description like '%rain%' OR weather_records.description like '%storm%' THEN 'Bad'
        ELSE 'Good'
        END AS weather_conditions
FROM
    weather_records

In [ ]:
-- 6. Recupere da tabela trips todas as corridas que começaram no Loop (pickup_location_id: 50) em um sábado e
-- terminaram em O'Hare (dropoff_location_id: 63). Obtenha as condições meteorológicas para cada corrida. 
-- Use o método que você aplicou na tarefa anterior. Além disso, recupere a duração de cada corrida. 
-- Ignore corridas para as quais os dados sobre as condições meteorológicas não estão disponíveis.

-- As colunas da tabela devem estar na seguinte ordem:

-- start_ts
-- weather_conditions
-- duration_seconds
-- Ordene por trip_id.

SELECT
    trips.start_ts AS start_ts,
    SUBQ.weather_conditions AS weather_conditions,
    trips.duration_seconds AS duration_seconds
FROM
    trips
INNER JOIN
    (SELECT
    weather_records.ts as ts,
    CASE
        WHEN weather_records.description like '%rain%' OR weather_records.description like '%storm%' THEN 'Bad'
        ELSE 'Good'
    END AS weather_conditions
FROM
    weather_records) AS SUBQ ON SUBQ.ts = trips.start_ts
WHERE
    trips.pickup_location_id = 50
    AND trips.dropoff_location_id = 63
    AND EXTRACT(DOW FROM trips.start_ts) = 6
ORDER BY
    trip_id